<a href="https://colab.research.google.com/github/4mami/SummarizeForEachRow/blob/main/summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 29.1 MB/s 
     |████████████████████████████████| 7.6 MB 53.7 MB/s 
     |████████████████████████████████| 182 kB 74.2 MB/s 


In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer, set_seed
from glob import glob
import torch
import tqdm

set_seed(1234)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

model = AutoModelWithLMHead.from_pretrained("t5-large")
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("t5-large")

device: cuda:0


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1136: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
# 1行が大体350 wordsぐらい(512トークン以下)のテキストファイルを読み込ませる
txt_files = glob('*.txt')
print(f"file name: {txt_files[0]}")

# テキストファイルを1行ずつ読んでいく
with open(txt_files[0]) as f:
  for line in tqdm.tqdm(f):
    line = line.rstrip()
    # その1行をトークナイズして、inputsを得る
    inputs = tokenizer.encode("summarize: " + line, return_tensors="pt")
    inputs = inputs.to(device)

    # inputsの長さを取得
    length = len(inputs[0])
    max_length = round(length * 0.2)
    min_length = round(length * 0.1)
    print(f"length: {length}, max_length: {max_length}, min_length: {min_length}")
    
    # 上の長さの10~20%になるようにoutputsを取得
    outputs = model.generate(inputs, max_length=max_length, min_length=min_length, num_beams=4, early_stopping=True)
    s = tokenizer.decode(outputs[0])
    
    # 出力用ファイルを開く
    with open("output.txt", "a") as output_f:
      # 出力用ファイルに1行分書き込む
      output_f.write(f"{s}\n")